In [1]:
host = "localhost" #"mucgpt-dev-db.postgres.database.azure.com"
database = "mucgpt" #"postgres"
username = "mucgpt-user" #"mucgpt_db_dev_admin"
password = "test" #"7nD6kLUVn7skmILuQs4t"

In [2]:

from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, Text
from datetime import datetime
Base = declarative_base()

class Requestinfo(Base):
    __tablename__ = 'requestinfo'

    id = Column(Integer(), primary_key=True)
    tokencount = Column(Integer())
    department = Column(String(20), nullable=False)
    messagecount = Column(Integer())
    method = Column(String(10))
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return '<ID %r, Department %r, Tokencount %r, Method %r, Messagecount %r> ' % (self.id, self.department, self.tokencount, self.method, self.messagecount)


In [69]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.orm import Session
from sqlalchemy import func


class Repository:
    def __init__(self, username: str, host: str, database: str, password: str):
        url = URL.create(
            drivername="postgresql",
            username=username,
            host=host,
            database=database,
            password=password
        )
        self.engine = create_engine(url)
    
    def setup_schema(self, base):
        base.metadata.create_all(self.engine)

    def addInfo(self, info: Requestinfo):
        with Session(self.engine) as session:
            session.add(info)
            session.commit()
    def get(self, id):
         with Session(self.engine) as session:
            return session.get(Requestinfo, id)
    def getAll(self):
        with Session(self.engine) as session:
            infos_objs = session.query(Requestinfo)
            results = infos_objs.all()
            return results

    
    def countByDepartment(self):
        with Session(self.engine) as session:
            queryResult = session.query(Requestinfo.department, func.count(Requestinfo.tokencount)).group_by(Requestinfo.department)
            results = queryResult.all()
            results = [tuple(row) for row in results]
            return results

    def sumByDepartment(self):
        with Session(self.engine) as session:
            queryResult = session.query(Requestinfo.department, func.sum(Requestinfo.tokencount)).group_by(Requestinfo.department)
            results = queryResult.all()
            results = [tuple(row) for row in results]
            return results

    def avgByDepartment(self):
        with Session(self.engine) as session:
            queryResult = session.query(Requestinfo.department, func.avg(Requestinfo.tokencount)).group_by(Requestinfo.department)
            results = queryResult.all()
            results = [tuple(row) for row in results]
            return results
    
    def clear(self):
        with Session(self.engine) as session:
            session.query(Requestinfo).delete()
            session.commit()


In [70]:
repoHelper = Repository(
    username=username,
    host=host,
    database=database,
    password=password
)

In [71]:
repoHelper.setup_schema(Base)

In [72]:
repoHelper.clear()

In [73]:
Requestinfo.__table__

Table('requestinfo', MetaData(), Column('id', Integer(), table=<requestinfo>, primary_key=True, nullable=False), Column('tokencount', Integer(), table=<requestinfo>), Column('department', String(length=20), table=<requestinfo>, nullable=False), Column('messagecount', Integer(), table=<requestinfo>), Column('method', String(length=10), table=<requestinfo>), Column('created_on', DateTime(), table=<requestinfo>, default=CallableColumnDefault(<function datetime.now at 0x000001C4FC504430>)), Column('updated_on', DateTime(), table=<requestinfo>, onupdate=CallableColumnDefault(<function datetime.now at 0x000001C4FC504550>), default=CallableColumnDefault(<function datetime.now at 0x000001C4FC5044C0>)), schema=None)

In [74]:
first = Requestinfo(
    tokencount = 200,
    department = "IL",
    messagecount= 2,
    method = "Chat")
second = Requestinfo(
    tokencount = 300,
    department = "KM8",
    messagecount= 1,
    method = "Chat")  
third = Requestinfo(
    tokencount = 153,
    department = "IL",
    messagecount= 2,
    method = "Chat")      

In [75]:
repoHelper.addInfo(first)
repoHelper.addInfo(second)
repoHelper.addInfo(third)

In [29]:
repoHelper.get(32)

In [30]:
repoHelper.countByDepartment()

[('KM8', 1), ('IL', 2)]

In [31]:
repoHelper.sumByDepartment()

[('KM8', 300), ('IL', 353)]

In [32]:
repoHelper.avgByDepartment()

[('KM8', Decimal('300.0000000000000000')),
 ('IL', Decimal('176.5000000000000000'))]

In [103]:
Requestinfo.__mapper__.columns[0].name

'id'

In [104]:
getattr(repoHelper.getAll()[0], Requestinfo.__mapper__.columns[0].name)

59

In [105]:
[Requestinfo.__mapper__.columns[0].name]

['id']

In [106]:
def getData(curr, columname):
    data = getattr(curr, columname)
    if isinstance(data, str):
        return data.encode()
    else:
        return data

In [121]:
import csv
import io

memfile = io.StringIO()
outcsv = csv.writer(memfile, delimiter=',',quotechar='"', quoting = csv.QUOTE_MINIMAL)
outcsv.writerow([column.name for column in Requestinfo.__mapper__.columns])
[outcsv.writerow([getattr(curr, column.name) for column in Requestinfo.__mapper__.columns]) for curr in repoHelper.getAll()]

memfile.seek(0)

# Das StringIO-Objekt in ein BytesIO-Objekt umwandeln

memfile_bytesio = io.BytesIO(memfile.getvalue().encode())
memfile_bytesio.getvalue()

b'id,tokencount,department,messagecount,method,created_on,updated_on\r\n59,200,IL,2,Chat,2024-01-02 18:23:12.154606,2024-01-02 18:23:12.154606\r\n60,300,KM8,1,Chat,2024-01-02 18:23:12.162276,2024-01-02 18:23:12.162276\r\n61,153,IL,2,Chat,2024-01-02 18:23:12.174699,2024-01-02 18:23:12.174699\r\n'

In [96]:
getData(repoHelper.getAll()[0], "created_on")

datetime.datetime(2024, 1, 2, 18, 23, 12, 154606)

In [100]:
[column.name.encode() for column in Requestinfo.__mapper__.columns]

[b'id',
 b'tokencount',
 b'department',
 b'messagecount',
 b'method',
 b'created_on',
 b'updated_on']